In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Return Forecasting: Read Historical Daily Yen Futures Data
In this notebook, you will load historical Dollar-Yen exchange rate futures data and apply time series analysis and modeling to determine whether there is any predictable behavior.

In [2]:
# Futures contract on the Yen-dollar exchange rate:
# This is the continuous chain of the futures contracts that are 1 month to expiration
yen_futures = pd.read_csv(
    Path("../Resources/yen.csv"), index_col="Date", infer_datetime_format=True, parse_dates=True
)
yen_futures.head()

FileNotFoundError: [Errno 2] No such file or directory: '../Resources/yen.csv'

In [ ]:
# Trim the dataset to begin on January 1st, 1990
yen_futures = yen_futures.loc["1990-01-01":, :]
yen_futures.head()

 # Return Forecasting: Initial Time-Series Plotting

 Start by plotting the "Settle" price. Do you see any patterns, long-term and/or short?

In [ ]:
# Plot just the "Settle" column from the dataframe:
# YOUR CODE HERE!
yen_futures['Settle'].plot.line(
    title = 'Yen Futures Settle Prices',
    figsize = (15, 8))
plt.savefig("../Resources/settle_yen.png")

---

# Decomposition Using a Hodrick-Prescott Filter

 Using a Hodrick-Prescott Filter, decompose the Settle price into a trend and noise.

In [ ]:
import statsmodels.api as sm

# Apply the Hodrick-Prescott Filter by decomposing the "Settle" price into two separate series:
# YOUR CODE HERE!
yf_noise, yf_trend = sm.tsa.filters.hpfilter(yen_futures['Settle'])

In [ ]:
# Create a dataframe of just the settle price, and add columns for "noise" and "trend" series from above:
# YOUR CODE HERE!
yen_hp_df = pd.DataFrame(yen_futures['Settle'])
yen_hp_df['Noise'] = yf_noise
yen_hp_df['Trend'] = yf_trend
yen_hp_df.head()

In [ ]:
# Plot the Settle Price vs. the Trend for 2015 to the present
# YOUR CODE HERE!
yen_hp_df[['Settle', 'Trend']].loc['2015':].plot.line(
    title = 'Settle vs. Trend', 
    figsize = (15, 8))
plt.savefig("../Resources/settle_v_trend.png")

In [ ]:
# Plot the Settle Noise
# YOUR CODE HERE!
yen_hp_df['Noise'].loc['2015':].plot.line(
    title = 'Noise',
    figsize = (15, 8))
plt.savefig("../Resources/noise.png")

---

# Forecasting Returns using an ARMA Model

Using futures Settle *Returns*, estimate an ARMA model

1. ARMA: Create an ARMA model and fit it to the returns data. Note: Set the AR and MA ("p" and "q") parameters to p=2 and q=1: order=(2, 1).
2. Output the ARMA summary table and take note of the p-values of the lags. Based on the p-values, is the model a good fit (p < 0.05)?
3. Plot the 5-day forecast of the forecasted returns (the results forecast from ARMA model)

In [ ]:
# Create a series using "Settle" price percentage returns, drop any nan"s, and check the results:
# (Make sure to multiply the pct_change() results by 100)
# In this case, you may have to replace inf, -inf values with np.nan"s
returns = (yen_futures[["Settle"]].pct_change() * 100)
returns = returns.replace(-np.inf, np.nan).dropna()
returns.tail()

In [ ]:
from statsmodels.tsa.arima_model import ARMA

# Estimate and ARMA model using statsmodels (use order=(2, 1))
# YOUR CODE HERE!
model = ARMA(returns['Settle'].values, order=(2, 1))
# Fit the model and assign it to a variable called results
# YOUR CODE HERE!
results = model.fit()

In [ ]:
# Output model summary results:
# YOUR CODE HERE!
results.summary()

In [ ]:
# Plot the 5 Day Returns Forecast
# YOUR CODE HERE!
pd.DataFrame(results.forecast(steps = 5)[0]).plot(
    title = '5 Day Returns Forecast',
    figsize = (15, 8))
plt.savefig("../Resources/arma.png")

---

# Forecasting the Settle Price using an ARIMA Model

 1. Using the *raw* Yen **Settle Price**, estimate an ARIMA model.
     1. Set P=5, D=1, and Q=1 in the model (e.g., ARIMA(df, order=(5,1,1))
     2. P= # of Auto-Regressive Lags, D= # of Differences (this is usually =1), Q= # of Moving Average Lags
 2. Output the ARIMA summary table and take note of the p-values of the lags. Based on the p-values, is the model a good fit (p < 0.05)?
 3. Construct a 5 day forecast for the Settle Price. What does the model forecast will happen to the Japanese Yen in the near term?

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

# Estimate and ARIMA Model:
# Hint: ARIMA(df, order=(p, d, q))
# YOUR CODE HERE!
arima_model = ARIMA(yen_futures['Settle'], order = (4,1,1))
# Fit the model
# YOUR CODE HERE!
arima_results = arima_model.fit()

In [ ]:
# Output model summary results:
arima_results.summary()

In [ ]:
# Plot the 5 Day Price Forecast
# YOUR CODE HERE!
pd.DataFrame(arima_results.forecast(steps = 5)[0]).plot(
    title = '5 Day Futures Price Forecast',
    figsize = (15, 8))
plt.savefig("../Resources/arima.png")

---

# Volatility Forecasting with GARCH

Rather than predicting returns, let's forecast near-term **volatility** of Japanese Yen futures returns. Being able to accurately predict volatility will be extremely useful if we want to trade in derivatives or quantify our maximum loss.
 
Using futures Settle *Returns*, estimate an GARCH model

1. GARCH: Create an GARCH model and fit it to the returns data. Note: Set the parameters to p=2 and q=1: order=(2, 1).
2. Output the GARCH summary table and take note of the p-values of the lags. Based on the p-values, is the model a good fit (p < 0.05)?
3. Plot the 5-day forecast of the volatility.

In [ ]:
from arch import arch_model

In [ ]:
# Estimate a GARCH model:
# YOUR CODE HERE!
model = arch_model(returns['Settle'], p = 2, q = 1)
# Fit the model
# YOUR CODE HERE!
results = model.fit()

In [ ]:
# Summarize the model results
# YOUR CODE HERE!
results.summary()

In [ ]:
# Find the last day of the dataset
last_day = returns.index.max().strftime('%Y-%m-%d')
last_day

In [ ]:
# Create a 5 day forecast of volatility
forecast_horizon = 5
# Start the forecast using the last_day calculated above
# YOUR CODE HERE!
forecasts = results.forecast(horizon = forecast_horizon, start = last_day)

In [ ]:
# Annualize the forecast
intermediate = np.sqrt(forecasts.variance.dropna() * 252)
intermediate.head()

In [ ]:
# Transpose the forecast so that it is easier to plot
final = intermediate.dropna().T
final.head()

In [ ]:
# Plot the final forecast
# YOUR CODE HERE!
final.plot(
    title = '5 Day Forecast of Volatility',
    figsize = (15, 8))
plt.savefig("../Resources/garch.png")

---

# Conclusions

Based on your time series analysis, would you buy the yen now?

Is the risk of the yen expected to increase or decrease?

Based on the model evaluation, would you feel confident in using these models for trading?